<a href="https://colab.research.google.com/github/Sanyamgam24/movie-recommendation-model/blob/main/projectmovie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("tmdb_5000_movies.csv")

print("First 5 rows of the dataset:")
display(df.head())

print("\nInformation about the dataset (columns, non-null counts, data types):")
df.info()

First 5 rows of the dataset:


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",10-12-2009,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",19-05-2007,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",26-10-2015,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",16-07-2012,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",07-03-2012,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124



Information about the dataset (columns, non-null counts, data types):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  s

### How to verify your file in Google Colab

1.  **Check the Files Panel**: On the left sidebar of your Colab notebook, click on the folder icon to open the "Files" panel. Make sure `tmdb_5000_movies.csv` is listed there.

2.  **Ensure Correct Path**: By default, `pd.read_csv()` looks for files in the same directory as your notebook. If you uploaded the file using the files panel, it should be in the correct place. However, sometimes there can be a slight name mismatch or the file might be in a subdirectory.

3.  **List Files to Confirm Name**: You can use a shell command in a code cell to list the files in your current directory and confirm the exact filename and its presence.

In [ ]:
import pandas as pd
import ast

# Assuming 'df' is already loaded as in the initial setup
# If not, you would load it here:
# df = pd.read_csv("tmdb_5000_movies.csv")

# Helper functions for parsing list-like strings
def parse_list(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)]
        except (ValueError, SyntaxError):
            return []
    return []

def parse_cast(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)[:3]] # Top 3 cast members
        except (ValueError, SyntaxError):
            return []
    return []

def parse_crew(x):
    if isinstance(x, str):
        try:
            crew = ast.literal_eval(x)
            for member in crew:
                if member['job'] == 'Director': # Only director is extracted
                    return [member['name']]
        except (ValueError, SyntaxError):
            pass
    return []

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
df = pd.read_csv("tmdb_5000_movies.csv")

# Helper functions for parsing list-like strings
def parse_list(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)]
        except (ValueError, SyntaxError):
            return []
    return []

def parse_cast(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)[:3]] # Top 3 cast members
        except (ValueError, SyntaxError):
            return []
    return []

def parse_crew(x):
    if isinstance(x, str):
        try:
            crew = ast.literal_eval(x)
            for member in crew:
                if member['job'] == 'Director': # Only director is extracted
                    return [member['name']]
        except (ValueError, SyntaxError):
            pass
    return [] # Ensure an empty list is returned if director not found or error

# Apply parsing functions to relevant columns
# The original CSV should have these columns, so this is mostly defensive.

if 'genres' in df.columns:
    df['genres'] = df['genres'].apply(parse_list)
else:
    # This case should ideally not be hit with the original dataset
    print("Warning: 'genres' column not found after loading. Initializing with empty lists.")
    df['genres'] = [[] for _ in range(len(df))]

if 'keywords' in df.columns:
    df['keywords'] = df['keywords'].apply(parse_list)
else:
    print("Warning: 'keywords' column not found after loading. Initializing with empty lists.")
    df['keywords'] = [[] for _ in range(len(df))]

if 'cast' in df.columns:
    df['cast'] = df['cast'].apply(parse_cast)
else:
    print("Warning: 'cast' column not found after loading. Initializing with empty lists.")
    df['cast'] = [[] for _ in range(len(df))]

if 'crew' in df.columns:
    df['crew'] = df['crew'].apply(parse_crew)
else:
    print("Warning: 'crew' column not found after loading. Initializing with empty lists.")
    df['crew'] = [[] for _ in range(len(df))]

# Fill NaN values in 'overview' with empty strings, as it's a text feature
df['overview'] = df['overview'].fillna("")

# Remove spaces from individual names within lists for better tag processing
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# Diagnostic: Print first few parsed genres to verify
print("\nFirst 5 parsed genres (after cleaning spaces):")
for i in range(min(5, len(df))):
    print(f"Movie {i} genres: {df['genres'].iloc[i]}")

# Diagnostic: Verify if 'Action' genre is present and count movies
action_movies_count = df['genres'].apply(lambda x: 'Action' in x).sum()
print(f"\nNumber of movies with 'Action' genre after parsing: {action_movies_count}")

# Create a "tags" column by combining all processed features
df['tags'] = df['overview'] + " " + \
             df['genres'].apply(lambda x: " ".join(x)) + " " + \
             df['keywords'].apply(lambda x: " ".join(x)) + " " + \
             df['cast'].apply(lambda x: " ".join(x)) + " " + \
             df['crew'].apply(lambda x: " ".join(x))

# Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(df['tags']).toarray()

similarity = cosine_similarity(vectors)

# Recommendation function
def recommend(movie_title):
    # Convert input movie title to lowercase for case-insensitive comparison
    lower_movie_title = movie_title.lower()

    # Find movies where the lowercase title matches the lowercase input
    matching_movies = df[df['title'].str.lower() == lower_movie_title]

    if matching_movies.empty:
        print("Movie not found!")
        return

    # If there are multiple matches, pick the first one
    index = matching_movies.index[0]
    original_title = df.loc[index, 'title'] # Get the original title for printing

    distances = similarity[index]

    movie_list = sorted(list(enumerate(distances)),
                        key=lambda x: x[1],
                        reverse=True)[1:6]

    print(f"\nTop 5 movies similar to '{original_title}':\n")
    for i in movie_list:
        print(df.iloc[i[0]].title)



First 5 parsed genres (after cleaning spaces):
Movie 0 genres: ['Action', 'Adventure', 'Fantasy', 'ScienceFiction']
Movie 1 genres: ['Adventure', 'Fantasy', 'Action']
Movie 2 genres: ['Action', 'Adventure', 'Crime']
Movie 3 genres: ['Action', 'Crime', 'Drama', 'Thriller']
Movie 4 genres: ['Action', 'Adventure', 'ScienceFiction']

Number of movies with 'Action' genre after parsing: 1154


### Handling Missing Columns during Feature Extraction

This code block explicitly checks for the presence of 'genres', 'keywords', 'cast', and 'crew' columns. If any of these columns are not found, they are initialized with empty lists for all rows. This ensures that the subsequent feature processing steps do not encounter errors due to missing data columns.

In [ ]:
# Apply parsing functions to relevant columns, handling potential missing columns
if 'genres' in df.columns:
    df['genres'] = df['genres'].apply(parse_list)
else:
    print("Warning: 'genres' column not found. Initializing with empty lists.")
    df['genres'] = [[] for _ in range(len(df))]

if 'keywords' in df.columns:
    df['keywords'] = df['keywords'].apply(parse_list)
else:
    print("Warning: 'keywords' column not found. Initializing with empty lists.")
    df['keywords'] = [[] for _ in range(len(df))]

if 'cast' in df.columns:
    df['cast'] = df['cast'].apply(parse_cast)
else:
    print("Warning: 'cast' column not found. Initializing with empty lists.")
    df['cast'] = [[] for _ in range(len(df))]

if 'crew' in df.columns:
    df['crew'] = df['crew'].apply(parse_crew)
else:
    print("Warning: 'crew' column not found. Initializing with empty lists.")
    df['crew'] = [[] for _ in range(len(df))]

# Fill NaN values in 'overview' with empty strings, as it's a text feature
df['overview'] = df['overview'].fillna("")

# The subsequent cleaning steps will operate on these (potentially initialized) columns
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

### Creating the 'tags' column for Vectorization

All the processed features are combined into a single string in the 'tags' column. This unified text is then used as input for the vectorization process, which converts text data into numerical vectors suitable for similarity calculations.

In [ ]:
# Combine all processed features into a single 'tags' column
df['tags'] = df['overview'] + " " + \
             df['genres'].apply(lambda x: " ".join(x)) + " " + \
             df['keywords'].apply(lambda x: " ".join(x)) + " " + \
             df['cast'].apply(lambda x: " ".join(x)) + " " + \
             df['crew'].apply(lambda x: " ".join(x))

print("First 5 entries of the 'tags' column (showing first 200 characters for brevity):\n")
for i in range(5):
    print(f"Movie {i}: {df['tags'].iloc[i][:200]}...")

First 5 entries of the 'tags' column (showing first 200 characters for brevity):

Movie 0: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.    ...
Movie 1: Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.    ...
Movie 2: A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to r...
Movie 3: Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Dep...
Movie 4: John Carter is a war-weary, former military captain who's inexplicably transported to the mysterio

### Text Vectorization and Similarity Calculation

This final step converts the 'tags' into numerical vectors using `CountVectorizer` and then calculates the cosine similarity between all movies. This similarity matrix is the core of the recommendation engine.

In [ ]:
# Initialize CountVectorizer to convert text into a matrix of token counts
# max_features=5000 limits the vocabulary to the 5000 most frequent words
# stop_words='english' removes common English stop words (e.g., 'the', 'a', 'is')
cv = CountVectorizer(max_features=5000, stop_words='english')

# Fit the vectorizer to the 'tags' column and transform the text into a sparse matrix of counts
vectors = cv.fit_transform(df['tags']).toarray()

print("Shape of the vectorized data (number of movies, number of features):", vectors.shape)

# Calculate the cosine similarity between each movie's vector
similarity = cosine_similarity(vectors)

print("Shape of the similarity matrix (number of movies x number of movies):", similarity.shape)

Shape of the vectorized data (number of movies, number of features): (4803, 5000)
Shape of the similarity matrix (number of movies x number of movies): (4803, 4803)


In [ ]:
def get_movies_by_genre(genre_name):
    # Convert input genre to lowercase and remove spaces for consistent comparison
    cleaned_genre = genre_name.replace(" ", "").lower()

    # Filter DataFrame to find movies where the cleaned genre is in the 'genres' list
    # The 'genres' column already has cleaned (space-removed) and lowercase items due to previous processing
    genre_movies = df[df['genres'].apply(lambda x: cleaned_genre in [g.lower() for g in x])]

    if genre_movies.empty:
        print(f"No movies found for the genre '{genre_name}'. Please check the spelling or try another genre.")
        return

    print(f"\nMovies in the '{genre_name}' genre:\n")
    for title in genre_movies['title'].head(10):
        print(title)

    if len(genre_movies) > 10:
        print(f"(Showing first 10 of {len(genre_movies)} movies in this genre)")

In [ ]:
user_genre_input = input("Enter a genre to find movies: ")
get_movies_by_genre(user_genre_input)


Enter a genre to find movies: action

Movies in the 'action' genre:

Avatar
Pirates of the Caribbean: At World's End
Spectre
The Dark Knight Rises
John Carter
Spider-Man 3
Avengers: Age of Ultron
Batman v Superman: Dawn of Justice
Superman Returns
Quantum of Solace
(Showing first 10 of 1154 movies in this genre)


In [ ]:
def get_user_movie_recommendation():
    user_input = input("Enter a movie title to get recommendations: ")
    recommend(user_input)

get_user_movie_recommendation()

Enter a movie title to get recommendations: cars

Top 5 movies similar to 'Cars':

Cars 2
McFarland, USA
Silverado
The Fog
Open Road


In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------------
# 1. LOAD DATASET
# ----------------------------------
df = pd.read_csv("tmdb_5000_movies.csv")

# ----------------------------------
# 2. HELPER FUNCTIONS
# ----------------------------------
def parse_list(x):
    return [i['name'] for i in ast.literal_eval(x)]

def parse_cast(x):
    return [i['name'] for i in ast.literal_eval(x)[:3]]

def parse_crew(x):
    crew = ast.literal_eval(x)
    for member in crew:
        if member['job'] == 'Director':
            return [member['name']]
    return []


# ----------------------------------
# 3. CLEANING FEATURES
# ----------------------------------
df['genres'] = df['genres'].apply(parse_list)
df['keywords'] = df['keywords'].apply(parse_list)
df['cast'] = df['cast'].apply(parse_cast)
df['crew'] = df['crew'].apply(parse_crew)
df['overview'] = df['overview'].fillna("")

# remove spaces from names (Brad Pitt → BradPitt)
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# ----------------------------------
# 4. CREATE "TAGS" COLUMN
# ----------------------------------
df['tags'] = df['overview'] + " " + \
             df['genres'].apply(lambda x: " ".join(x)) + " " + \
             df['keywords'].apply(lambda x: " ".join(x)) + " " + \
             df['cast'].apply(lambda x: " ".join(x)) + " " + \
             df['crew'].apply(lambda x: " ".join(x))

# ----------------------------------
# 5. TF-IDF VECTORIZATION
# ----------------------------------
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
vectors = tfidf.fit_transform(df['tags']).toarray()

# similarity matrix
similarity = cosine_similarity(vectors)

# ----------------------------------
# 6. RECOMMENDATION FUNCTION
# ----------------------------------
def recommend(movie_title):
    movie_title = movie_title.strip()

    if movie_title not in df['title'].values:
        print("Movie not found!")
        return

    index = df[df['title'] == movie_title].index[0]
    distances = similarity[index]

    movie_list = sorted(
        list(enumerate(distances)),
        key=lambda x: x[1],
        reverse=True
    )[1:6]

    print(f"\nTop 5 movies similar to '{movie_title}':\n")
    for i in movie_list:
        print(df.iloc[i[0]].title)



recommend("The Dark Knight")


# Task
Refactor the movie recommendation system by consolidating all necessary imports, data loading, helper functions for parsing movie data, feature cleaning steps, tag creation, TF-IDF vectorization, the `recommend` function for movie title-based suggestions, and the `get_movies_by_genre` function for genre-based movie listings into a single, well-organized code cell. Afterward, demonstrate the usage of both the `recommend` and `get_movies_by_genre` functions with example inputs to verify their functionality. The data required is "tmdb_5000_movies.csv".

## Consolidate and Refactor Code

### Subtask:
Combine all necessary imports, data loading, helper functions, feature cleaning, tag creation, TF-IDF vectorization, similarity calculation, movie recommendation function, and the genre-based movie listing function into a single code cell, organized with clear headings.


**Reasoning**:
The subtask requires consolidating all the necessary code components (imports, data loading, helper functions, feature engineering, vectorization, similarity calculation, and recommendation functions) into a single code cell, organized with clear headings. This step will achieve that consolidation.



In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------------
# 1. LOAD DATASET
# ----------------------------------
df = pd.read_csv("tmdb_5000_movies.csv")

# ----------------------------------
# 2. HELPER FUNCTIONS
# ----------------------------------
def parse_list(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)]
        except (ValueError, SyntaxError):
            return []
    return []

def parse_cast(x):
    if isinstance(x, str):
        try:
            return [i['name'] for i in ast.literal_eval(x)[:3]] # Top 3 cast members
        except (ValueError, SyntaxError):
            return []
    return []

def parse_crew(x):
    if isinstance(x, str):
        try:
            crew = ast.literal_eval(x)
            for member in crew:
                if member['job'] == 'Director': # Only director is extracted
                    return [member['name']]
        except (ValueError, SyntaxError):
            pass
    return [] # Ensure an empty list is returned if director not found or error

# ----------------------------------
# 3. CLEANING FEATURES
# ----------------------------------
# Apply parsing functions to relevant columns, handling potential missing columns
if 'genres' in df.columns:
    df['genres'] = df['genres'].apply(parse_list)
else:
    print("Warning: 'genres' column not found. Initializing with empty lists.")
    df['genres'] = [[] for _ in range(len(df))]

if 'keywords' in df.columns:
    df['keywords'] = df['keywords'].apply(parse_list)
else:
    print("Warning: 'keywords' column not found. Initializing with empty lists.")
    df['keywords'] = [[] for _ in range(len(df))]

if 'cast' in df.columns:
    df['cast'] = df['cast'].apply(parse_cast)
else:
    print("Warning: 'cast' column not found. Initializing with empty lists.")
    df['cast'] = [[] for _ in range(len(df))]

if 'crew' in df.columns:
    df['crew'] = df['crew'].apply(parse_crew)
else:
    print("Warning: 'crew' column not found. Initializing with empty lists.")
    df['crew'] = [[] for _ in range(len(df))]

# Fill NaN values in 'overview' with empty strings, as it's a text feature
df['overview'] = df['overview'].fillna("")

# Remove spaces from individual names within lists for better tag processing
df['genres'] = df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
df['keywords'] = df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
df['cast'] = df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
df['crew'] = df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# ----------------------------------
# 4. CREATE "TAGS" COLUMN
# ----------------------------------
df['tags'] = df['overview'] + " " + \
             df['genres'].apply(lambda x: " ".join(x)) + " " + \
             df['keywords'].apply(lambda x: " ".join(x)) + " " + \
             df['cast'].apply(lambda x: " ".join(x)) + " " + \
             df['crew'].apply(lambda x: " ".join(x))

# ----------------------------------
# 5. TF-IDF VECTORIZATION AND SIMILARITY CALCULATION
# ----------------------------------
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
vectors = tfidf.fit_transform(df['tags']).toarray()
similarity = cosine_similarity(vectors)

# ----------------------------------
# 6. RECOMMENDATION FUNCTION
# ----------------------------------
def recommend(movie_title):
    lower_movie_title = movie_title.lower()
    matching_movies = df[df['title'].str.lower() == lower_movie_title]

    if matching_movies.empty:
        print("Movie not found!")
        return

    index = matching_movies.index[0]
    original_title = df.loc[index, 'title']

    distances = similarity[index]

    movie_list = sorted(list(enumerate(distances)),
                        key=lambda x: x[1],
                        reverse=True)[1:6]

    print(f"\nTop 5 movies similar to '{original_title}':\n")
    for i in movie_list:
        print(df.iloc[i[0]].title)

def get_user_movie_recommendation():
    user_input = input("Enter a movie title to get recommendations: ")
    recommend(user_input)

# ----------------------------------
# 7. GENRE-BASED MOVIE LISTING FUNCTION
# ----------------------------------
def get_movies_by_genre(genre_name):
    cleaned_genre = genre_name.replace(" ", "").lower()
    genre_movies = df[df['genres'].apply(lambda x: cleaned_genre in [g.lower() for g in x])]

    if genre_movies.empty:
        print(f"No movies found for the genre '{genre_name}'. Please check the spelling or try another genre.")
        return

    print(f"\nMovies in the '{genre_name}' genre:\n")
    for title in genre_movies['title'].head(10):
        print(title)

    if len(genre_movies) > 10:
        print(f"(Showing first 10 of {len(genre_movies)} movies in this genre)")

print("All necessary components for the movie recommendation system have been consolidated and initialized.")
print("You can now call 'recommend(movie_title)', 'get_user_movie_recommendation()', or 'get_movies_by_genre(genre_name)'.")

All necessary components for the movie recommendation system have been consolidated and initialized.
You can now call 'recommend(movie_title)', 'get_user_movie_recommendation()', or 'get_movies_by_genre(genre_name)'.


In [ ]:
get_user_movie_recommendation()

Enter a movie title to get recommendations: cars

Top 5 movies similar to 'Cars':

Cars 2
The Fast and the Furious: Tokyo Drift
McFarland, USA
The Fast and the Furious
Little Miss Sunshine


In [ ]:
print("\n--- Demonstrating get_movies_by_genre for 'action' ---")
get_movies_by_genre("action")


--- Demonstrating get_movies_by_genre for 'action' ---

Movies in the 'action' genre:

Avatar
Pirates of the Caribbean: At World's End
Spectre
The Dark Knight Rises
John Carter
Spider-Man 3
Avengers: Age of Ultron
Batman v Superman: Dawn of Justice
Superman Returns
Quantum of Solace
(Showing first 10 of 1154 movies in this genre)
